In [1]:
import requests
import json
from collections import Counter, defaultdict
from user_agent import generate_user_agent
import numpy as np
import multiprocessing as multi
from torrequest import TorRequest

In [2]:
#Using TOR to generate a random IP
with TorRequest() as tr:
    headers = {'User-Agent': 'bot 0.1'}   
    page_link = "https://www.courtlistener.com/api/rest/v3/opinions/"
    page_response = tr.get(page_link, headers=headers)
    try:
        if page_response.status_code == 200:
            number_of_opinions = page_response.json()["count"]
        else:
            print(page_response.status_code)
            print(str(requests.Timeout))
    except requests.Timeout as e:
        print("It is time to timeout")
        print(str(e))
    tr.reset_identity()

In [3]:
def chunks(n, page_list):
    """Splits the list into n chunks"""
    return np.array_split(page_list,n)
cpus = multi.cpu_count()
workers = []
page_list = ['https://www.courtlistener.com/api/rest/v3/opinions/%i' %i for i in range(number_of_opinions)]
page_bins = chunks(cpus, page_list)

In [9]:
def perform_extraction(page_ranges):
    #Mine all the opinions from the court listener API
    opinions = defaultdict()
    for page in page_ranges:
        print(page)
        page_response = requests.get(page)
        try:
            if page_response.status_code == 200:
                json = page_response.json()
                opinions[json["resource_uri"]] = json["opinions_cited"]
            else:
                print(page_response.status_code)
                print(str(requests.Timeout))
        except requests.Timeout as e:
            print("It is time to timeout")
            print(str(e))
        file_name = multi.current_process().name+'.txt'
        f= open(file_name,"w+")
        f.write(str(opinions))
        f.close() 

In [10]:
for cpu in range(cpus):
    worker = multi.Process(name=str(cpu), 
                           target=perform_extraction, 
                           args=(page_bins[cpu],))
    worker.start()
    workers.append(worker)

for worker in workers:
    worker.join()

https://www.courtlistener.com/api/rest/v3/opinions/0
https://www.courtlistener.com/api/rest/v3/opinions/1021684
https://www.courtlistener.com/api/rest/v3/opinions/2043368
https://www.courtlistener.com/api/rest/v3/opinions/3065052
404
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/1
https://www.courtlistener.com/api/rest/v3/opinions/1021685
https://www.courtlistener.com/api/rest/v3/opinions/2043369
https://www.courtlistener.com/api/rest/v3/opinions/3065053
https://www.courtlistener.com/api/rest/v3/opinions/3065054
https://www.courtlistener.com/api/rest/v3/opinions/1021686
https://www.courtlistener.com/api/rest/v3/opinions/2043370
https://www.courtlistener.com/api/rest/v3/opinions/2
https://www.courtlistener.com/api/rest/v3/opinions/1021687
https://www.courtlistener.com/api/rest/v3/opinions/3065055
https://www.courtlistener.com/api/rest/v3/opinions/2043371
https://www.courtlistener.com/api/rest/v3/opinions/2043372
https://www.courtlistener.com/ap

429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/27
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/1021717
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/2043400
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/3065080
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/28
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/1021718
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/2043401
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/3065081
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/29
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/1021719
429
<

https://www.courtlistener.com/api/rest/v3/opinions/3065100
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/2043420
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/1021738
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/48
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/3065101
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/2043421
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/1021739
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/49
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/3065102
429
<class 'requests.exceptions.Timeout'>
https://www.courtlistener.com/api/rest/v3/opinions/2043422
429
<class 'requests.exceptions.Timeout'>


Process 0:
Process 1:
Process 3:
Traceback (most recent call last):
Process 2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse(buffering=True)
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/mu

  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/requests/sessions.py", line 218, in resolve_redirects
    **adapter_kwargs
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/urllib3/connectionpool.py", line 850, in _validate_conn
    conn.connect()
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/urllib3/util/ssl_.py", line 329, in ssl_wrap_socket
    return context.wrap_socket(sock, server_hostname=server_hostname)
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/urllib3/connection.py", line 326, in connect
    ssl_context=context)
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/ssl.py", line 407, in wrap_socket
    _context=self, _session=session)
  File "/usr/local/Cellar/python3/3.6.4_2/Fram

KeyboardInterrupt: 